# Assignment 3: Build a seq2seq model for machine translation.


### Name: SANJEET VINDO JAIN
### CWID: 20012768


### Task: Change LSTM model to Bidirectional LSTM Model and Translate English to Spanish

### Due Date: Wednesday, April 19th, 11:59PM

## 0. You will do the following:

1. Read and run the code. Please make sure you have installed keras or tensorflow.Running the script on colab will speed up the training process and also prevent package loading issue. 
2. Complete the code in Section 1.1, you may fill in your data directory.
3. Directly modify the code in Section 3. Change the current LSTM layer to a Bidirectional LSTM Model.
4. Training your model and translate English to Spanish in Section 4.2. You could try translating other languages.
5. Complete the code in Section 5.

### Hint: 

To implement ```Bi-LSTM```, you will need the following code to build the encoder **in Section 3**. Do NOT use Bi-LSTM for the decoder. But there are other codes **you need to modify** to make it work.

In [1]:
# from keras.layers import Bidirectional, Concatenate

# encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
#                                   dropout=0.5, name='encoder_lstm'))
# _, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])

## 1. Data preparation (10 points)

1. Download spanish-english data from http://www.manythings.org/anki/
2. You may try to use other languages.
3. Unzip the .ZIP file.
4. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".
5. Fill in your data directory in section 1.1.

In [2]:
#just checking my tensor flow setup with my gpu
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### 1.1. Load and clean text


In [3]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [4]:
# e.g., filename = 'Data/deu.txt'
filename = "spa.txt"

# e.g., n_train = 20000
n_train = 20000

In [5]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [6]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[were young] => [somos jovenes]
[weve eaten] => [hemos comido]
[what a bore] => [que aburrimiento]
[what a dope] => [que burro eres]
[what a dope] => [que burro]
[what a heel] => [que tipo tan arrastrado]
[what a jerk] => [que pendejo]
[what a jerk] => [que imbecil]
[what a jerk] => [que cretino]
[what a life] => [que vida]


In [7]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (20000,)
Length of target_texts: (20000,)


In [8]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 48


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [9]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
# using new keras version so the import changed
from keras_preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 48)
shape of target_token_index: 29


In [10]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [11]:
target_texts[100]

'\tsali\n'

In [12]:
decoder_input_seq[100, :]

array([ 6,  5,  4, 12, 11,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [13]:
from tensorflow import keras
from keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 48, 30)


## 3. Build the networks (for training) (20 points)

- In this section, we have already implemented the LSTM model for you. You can run the code and see what the code is doing.  

- **You need to change the existing LSTM model to a Bidirectional LSTM model. Just modify the network structrue and do not change the training cell in section 3.4.**

- Build encoder, decoder, and connect the two modules to get "model". 

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.



### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [14]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, LSTM

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# set the LSTM layer
encoder_lstm = LSTM(latent_dim, return_state=True, 
                    dropout=0.5, name='encoder_lstm')
_, state_h, state_c = encoder_lstm(encoder_inputs)

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')

Print a summary and save the encoder network structure to "./encoder.pdf"

In [15]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model
# fixed a path issue with windows 11 with reference from : https://stackoverflow.com/questions/40632486/dot-exe-not-found-in-path-pydot-on-python-windows-7 
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz/bin'
SVG(model_to_dot(encoder_model, show_shapes=False,).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_inputs (InputLayer)  [(None, None, 28)]       0         
                                                                 
 encoder_lstm (LSTM)         [(None, 256),             291840    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 291,840
Trainable params: 291,840
Non-trainable params: 0
_________________________________________________________________


### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [16]:
from tensorflow import keras 
from keras.layers import Input, LSTM, Dense
from tensorflow import keras 
from keras.models import Model

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

Print a summary and save the encoder network structure to "./decoder.pdf"

In [17]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 decoder_input_h (InputLayer)   [(None, 256)]        0           []                               
                                                                                                  
 decoder_input_c (InputLayer)   [(None, 256)]        0           []                               
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 256),  293888      ['decoder_input_x[0][0]',        
                                 (None, 256),                     'decoder_input_h[0][0]',  

### 3.3. Connect the encoder and decoder

In [18]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [19]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_x (InputLayer)   [(None, None, 28)]   0           []                               
                                                                                                  
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 encoder (Functional)           [(None, 256),        291840      ['encoder_input_x[0][0]']        
                                 (None, 256)]                                                     
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 256),  293888      ['decoder_input_x[0]

### 3.4. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [20]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 48, 30)
shape of decoder_target_data(20000, 48, 30)


In [21]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50
250/250 [==============================] - 9s 18ms/step - loss: 1.0397 - val_loss: 0.9695
Epoch 2/50
250/250 [==============================] - 4s 15ms/step - loss: 0.8352 - val_loss: 0.8641
Epoch 3/50
250/250 [==============================] - 4s 15ms/step - loss: 0.7970 - val_loss: 0.8164
Epoch 4/50
250/250 [==============================] - 4s 15ms/step - loss: 0.7730 - val_loss: 0.7891
Epoch 5/50
250/250 [==============================] - 4s 15ms/step - loss: 0.7537 - val_loss: 0.7561
Epoch 6/50
250/250 [==============================] - 4s 15ms/step - loss: 0.7396 - val_loss: 0.7437
Epoch 7/50
250/250 [==============================] - 4s 15ms/step - loss: 0.7270 - val_loss: 0.7211
Epoch 8/50
250/250 [==============================] - 4s 14ms/step - loss: 0.7161 - val_loss: 0.7066
Epoch 9/50
250/250 [==============================] - 4s 15ms/step - loss: 0.7068 - val_loss: 0.6904
Epoch 10/50
250/250 [==============================] - 4s 15ms/step - loss: 0.6981 - val_lo

## 4. Make predictions

- In this section, you need to complete section 4.2 to translate English to the target language.




### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [22]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [23]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [24]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


1/1 [==============================] - 0s 15ms/step
-
English:        dont smoke
Spanish (true):  no fumeis
Spanish (pred):  no se lostis
1/1 [==============================] - 0s 16ms/step
-
English:        dont smoke
Spanish (true):  no fumais
Spanish (pred):  no se lostis
1/1 [==============================] - 0s 15ms/step
-
English:        dont speak
Spanish (true):  no hables
Spanish (pred):  no se suentes
1/1 [==============================] - 0s 16ms/step
-
English:        dont worry
Spanish (true):  no te preocupes
Spanish (pred):  no se lostis
1/1 [==============================] - 0s 15ms/step
-
English:        dont worry
Spanish (true):  no os preocupeis
Spanish (pred):  no se lostis
1/1 [==============================] - 0s 15ms/step
-
English:        dont worry
Spanish (true):  no se preocupen
Spanish (pred):  no se lostis
1/1 [==============================] - 0s 15ms/step
-
English:        finish this
Spanish (true):  termine esto
Spanish (pred):  tenta esto
1/1 [=======

### 4.2. Translate an English sentence to the target language （20 points）

1. Tokenization
2. One-hot encode
3. Translate

In [25]:
input_sentence = ['I love you']

input_sequence, l = text2sequences(max_encoder_seq_length,input_sentence)

input_x = onehot_encode(input_sequence, max_encoder_seq_length, num_encoder_tokens)

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + str(input_sentence))
print('translated sentence is: ' + str(translated_sentence))

1/1 [==============================] - 0s 16ms/step
source sentence is: ['I love you']
translated sentence is: lo conozco



# 5. Evaluate the translation using BLEU score

- We have already translated from English to target language, but how can we evaluate the performance of our model quantitatively? 

- In this section, you need to re-train the model we built in secton 3 and then evaluate the bleu score on testing dataset.

Reference:

https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

https://en.wikipedia.org/wiki/BLEU

#### Hint:

- Randomly partition the dataset to training, validation, and test.

- Evaluate the BLEU score using the test set. Report the average.

- You may use packages to calculate bleu score, e.g., sentence_bleu() from nltk package.

### 5.1. Partition the dataset to training, validation, and test. Build new token index. (10 points)

1. You may try to load more data/lines from text file.
2. Convert text to sequences and build token index using training data.
3. One-hot encode your training and validation text sequences.

In [26]:
filename = 'spa.txt'

# e.g., n_train = 30000
n_train = 50000

# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]
input_texts = clean_pairs[:,0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

In [27]:
rand_indices = numpy.random.permutation(50000)
train_indices = rand_indices[0:int(50000*.98)]
test_indices = rand_indices[int(50000*.98):int(50000)]

input_train = input_texts[train_indices]
# input_valid = input_texts[valid_indices]
input_test = input_texts[test_indices]

target_train = numpy.asarray(target_texts)[train_indices]
# target_valid = numpy.asarray(target_texts)[valid_indices]
target_test = numpy.asarray(target_texts)[test_indices]

In [28]:
encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_train)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_train)



print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (49000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (49000, 48)
shape of target_token_index: 29


In [29]:
encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(49000, 18, 28)
(49000, 48, 30)


### 5.2 Retrain your previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score. (25 points)

1. Use the model structure in section 3 to train a new model with new training and validation datasets.
2. Based on validation BLEU score or loss to tune parameters.

In [30]:
model.load_weights("seq2seq.h5")
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=100, epochs=50)


Epoch 1/50
490/490 [==============================] - 8s 13ms/step - loss: 1.0645
Epoch 2/50
490/490 [==============================] - 6s 12ms/step - loss: 0.9398
Epoch 3/50
490/490 [==============================] - 6s 12ms/step - loss: 0.9070
Epoch 4/50
490/490 [==============================] - 6s 12ms/step - loss: 0.8871
Epoch 5/50
490/490 [==============================] - 6s 12ms/step - loss: 0.8717
Epoch 6/50
490/490 [==============================] - 6s 12ms/step - loss: 0.8595
Epoch 7/50
490/490 [==============================] - 6s 13ms/step - loss: 0.8493
Epoch 8/50
490/490 [==============================] - 6s 12ms/step - loss: 0.8412
Epoch 9/50
490/490 [==============================] - 6s 13ms/step - loss: 0.8359
Epoch 10/50
490/490 [==============================] - 6s 12ms/step - loss: 0.8288
Epoch 11/50
490/490 [==============================] - 6s 13ms/step - loss: 0.8232
Epoch 12/50
490/490 [==============================] - 6s 12ms/step - loss: 0.8185
Epoch 13/50
4

### 5.3 Evaluate the BLEU score using the test set. (15 points)

1. Use trained model above to calculate the BLEU score with testing dataset.
2. A reasonable should be 0.1-0.3. The higher, the better.

In [31]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothIt = SmoothingFunction().method2
bleu_list = []
for n in range(len(input_test)):
  
  test_string = input_test[n]
  target = target_test[n]
  input = [test_string]

  encoder_input_seq,l= text2sequences(max_encoder_seq_length,input)
  input_x = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
  
  #it was showing some keyErrors because we used a smoothing function

  try:
    translated = decode_sequence(input_x)
  except KeyError as err:
    continue
  score = sentence_bleu(target, translated, smoothing_function = smoothIt)

  print(score)
  bleu_list.append(score)

1/1 [==============================] - 0s 17ms/step
0.1623339577375495
1/1 [==============================] - 0s 16ms/step
0.10694730137685537
1/1 [==============================] - 0s 16ms/step
0.4854917717073234
1/1 [==============================] - 0s 17ms/step
0.07454480401697919
1/1 [==============================] - 0s 15ms/step
0.09224978011409782
1/1 [==============================] - 0s 16ms/step
0.42044820762685725
1/1 [==============================] - 0s 16ms/step
0.17657994853073264
1/1 [==============================] - 0s 17ms/step
0.537284965911771
1/1 [==============================] - 0s 15ms/step
0.18221085232151557
1/1 [==============================] - 0s 15ms/step
0.2730120862709067
1/1 [==============================] - 0s 15ms/step
0.33980884896942454
1/1 [==============================] - 0s 15ms/step
0.537284965911771
1/1 [==============================] - 0s 16ms/step
0.13950796967929135
1/1 [==============================] - 0s 22ms/step
0.10768746526902671

In [32]:
print('BLEU score is:',numpy.mean(bleu_list))

BLEU score is: 0.2811591285480474
